In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

# Для работы с матрицами
from scipy.sparse import csr_matrix

import implicit

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k

In [2]:
data = pd.read_csv('../../data/rec_sys/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../../data/rec_sys/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

Количество департаментов 44. Не так много. Будем арскрашивать по департаментам.

In [5]:
item_features.department.nunique()

44

In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Для последовательности обучаем на 5000 товарах, но без проблем можно обучить и на всех 90 тыс.

In [7]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [8]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Максимальное количество покупок

In [9]:
user_item_matrix.max().max()

2926.0

In [10]:
user_item_matrix.shape

(2499, 5001)

In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
#?AlternatingLeastSquares

In [13]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, #
                                num_threads=4) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   
                        N=5, 
                        filter_already_liked_items=[itemid_to_id[999999]], 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 2.81 s


Для повышения качества нужно либо менять веса, лио тюнить гиперпараметры.

Посмотрим как выглядят рекомендации. В большинстве случаев от 0 до 1, но могут и выходить за эти интервалы.

In [14]:
recs

[(662, 0.8274390468197814),
 (2148, 0.7896217527266514),
 (215, 0.7374920167168156),
 (2927, 0.7262110765134022),
 (2307, 0.6215299512309506)]

In [15]:
[id_to_itemid[rec[0]] for rec in recs]

[866227, 981760, 834484, 1044078, 995242]

Функция конвертации прогноза модели в нужные item id

In [16]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,  
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    recalculate_user=True)]
    return res

In [17]:
result

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
...,...,...
2037,2496,[6534178]
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813..."


Запускаем ALS на всех юзерах и измеряем пресижн

In [18]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 31.8 s


0.1785504407443659

In [19]:
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.1785504407443659

С ходу без подборов гиперпараметров пресижн = 18% - выше, чем в моделях прошлого вебинара.

In [20]:
result.head(2)

,user_id,actual,als
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 1005186, 995242, 1105488]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5568378, 9337581, 1133018, 965766]"


BM25 взвешивание

In [21]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  

In [23]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

result['als_bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25'], row['actual']), axis=1).mean()


Wall time: 39.6 s


0.22223310479921352

In [24]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='nunique', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  

In [26]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

result['als_bm25_nunique'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_nunique'], row['actual']), axis=1).mean()


Wall time: 43.3 s


0.21439764936336686

Вариант весов 2: mean

In [27]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='mean', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  

In [29]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

result['als_bm25_mean'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_mean'], row['actual']), axis=1).mean()


Wall time: 38.9 s


0.2114593535749237

Вариант весов 3: quantity

In [30]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
user_item_matrix = bm25_weight(user_item_matrix.T).T   

In [32]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

result['als_bm25_quantity'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_quantity'], row['actual']), axis=1).mean()


Wall time: 54.5 s


0.1828599412340817

Вариант весов 3: size

In [33]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='size', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) 

sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  

In [35]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=8) 

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
          show_progress=True)

result['als_bm25_size'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

result.apply(lambda row: precision_at_k(row['als_bm25_size'], row['actual']), axis=1).mean()


Wall time: 40.8 s


0.21919686581782297

In [36]:
result

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 1005186, 995242, 1105488]","[1082185, 995242, 934369, 824546, 1051211]","[1082185, 1135694, 979707, 1005186, 958046]","[1082185, 986947, 986912, 995242, 1056509]","[1082185, 6534178, 995242, 981760, 1029743]","[995242, 1082185, 934369, 1100972, 1053282]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5568378, 9337581, 1133018, 965766]","[1092026, 951590, 1053690, 1133018, 1106523]","[951590, 826249, 910032, 1092026, 9337581]","[1082185, 6534178, 1029743, 1053690, 981760]","[6534178, 929668, 1082185, 1106523, 995242]","[1053690, 1133018, 1092026, 951590, 932298]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1023720, 1007195, 1051516, 866211, 986912]","[1082185, 1024306, 878996, 1023720, 934639]","[1082185, 878996, 1024306, 1127831, 1023720]","[1082185, 6534178, 1127831, 866211, 981760]","[1023720, 1082185, 878996, 1024306, 965267]","[1024306, 1082185, 1023720, 878996, 1051516]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[938700, 1082185, 1106523, 826249, 883404]","[1082185, 6944571, 849843, 893018, 1121393]","[1082185, 873527, 828857, 1127831, 1118627]","[1052046, 1013321, 857390, 911812, 1082185]","[1082185, 6534178, 1029743, 995242, 1106523]","[1058997, 1082185, 6944571, 849843, 1126899]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[844179, 1004906, 826249, 1053690, 938700]","[883404, 965766, 1004906, 1082185, 840361]","[1082185, 1068719, 840361, 1053690, 1004906]","[1082185, 995242, 6534178, 1133018, 1029743]","[6534178, 1082185, 995242, 1029743, 981760]","[1068719, 840361, 1004906, 883404, 1082185]"
...,...,...,...,...,...,...,...,...
2037,2496,[6534178],"[914190, 1133018, 1077555, 1013321, 1106523]","[914190, 904360, 1133018, 840361, 826249]","[914190, 1070702, 904360, 1000753, 995785]","[6534178, 1082185, 1133018, 981760, 995785]","[1082185, 940947, 6534178, 1133018, 995242]","[914190, 840361, 831628, 995785, 904360]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[5569230, 826249, 5569845, 1098066, 951590]","[5569230, 957951, 826249, 5569845, 995785]","[5569230, 981760, 951590, 1082185, 995785]","[5569230, 1082185, 6534178, 995242, 1029743]","[5569230, 5569471, 1044068, 1130858, 1022981]","[957951, 5569230, 981760, 995785, 5569845]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[1053690, 6534178, 862349, 1070820, 1082185]","[840361, 995242, 862349, 916381, 861272]","[840361, 824555, 951526, 904023, 1036432]","[6533889, 202291, 6513604, 7441391, 1052335]","[923670, 6533889, 202291, 999639, 1055853]","[862349, 995242, 840361, 6534178, 861272]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1004906, 6534178, 1098066, 1106523, 930917]","[826249, 883404, 1096036, 6534178, 5569845]","[826249, 930917, 883404, 1098066, 5569230]","[6534178, 6533889, 1082185, 995242, 5569230]","[6534178, 6533889, 1082185, 1029743, 995242]","[826249, 5569845, 1098066, 1053690, 883404]"


In [40]:
%%time

for factors in tqdm(np.linspace(30,170,8)):
    for reg in np.geomspace(0.0001,0.01,8):

        data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

        user_item_matrix = pd.pivot_table(data_train, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', 
                                          aggfunc='count', 
                                          fill_value=0
                                         )

        user_item_matrix = user_item_matrix.astype(float) 

        # переведем в формат saprse matrix
        sparse_user_item = csr_matrix(user_item_matrix).tocsr()

        user_item_matrix.head(3)

        user_item_matrix = bm25_weight(user_item_matrix.T).T  

        

        model = AlternatingLeastSquares(factors=int(factors), 
                                        regularization=reg,
                                        iterations=15, 
                                        calculate_training_loss=True, 
                                        num_threads=8)

        model.fit(csr_matrix(user_item_matrix).T.tocsr(),  
                  show_progress=False)

        result[f'als_bm25_factors={factors}_reg={reg}'] = \
                        result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

        result.apply(lambda row: precision_at_k(row[f'als_bm25_factors={factors}_reg={reg}'], \
                                                            row['actual']), axis=1).mean()

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

C:\Users\Alex\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
100%|███████████████████████████████████████████████████████████████████████████████████| 8/8 [40:19<00:00, 302.46s/it]


In [41]:
result

,user_id,actual,als,als_bm25,als_bm25_nunique,als_bm25_mean,als_bm25_quantity,als_bm25_size,als_bm25_factors=30.0_reg=0.0001,als_bm25_factors=30.0_reg=0.00019306977288832496,...,als_bm25_factors=150.0_reg=0.005179474679231213,als_bm25_factors=150.0_reg=0.01,als_bm25_factors=170.0_reg=0.0001,als_bm25_factors=170.0_reg=0.00019306977288832496,als_bm25_factors=170.0_reg=0.0003727593720314938,als_bm25_factors=170.0_reg=0.0007196856730011522,als_bm25_factors=170.0_reg=0.0013894954943731374,als_bm25_factors=170.0_reg=0.0026826957952797246,als_bm25_factors=170.0_reg=0.005179474679231213,als_bm25_factors=170.0_reg=0.01
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5569374, 1033142, 1005186, 995242, 1105488]","[1082185, 995242, 934369, 824546, 1051211]","[1082185, 1135694, 979707, 1005186, 958046]","[1082185, 986947, 986912, 995242, 1056509]","[1082185, 6534178, 995242, 981760, 1029743]","[995242, 1082185, 934369, 1100972, 1053282]","[1082185, 862349, 995242, 977658, 1025641]","[1082185, 995242, 977658, 1100972, 885290]",...,"[1082185, 995242, 1051211, 1056509, 965766]","[1082185, 995242, 965766, 934369, 1033142]","[995242, 1082185, 1033142, 965766, 9527290]","[965766, 1082185, 995242, 934369, 9527290]","[1082185, 995242, 965766, 1033142, 934369]","[1082185, 995242, 1051211, 1033142, 934369]","[1082185, 995242, 9527290, 867188, 1056509]","[1082185, 995242, 965766, 934369, 856942]","[995242, 1082185, 965766, 934369, 9527290]","[1082185, 995242, 965766, 1033142, 856942]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1106523, 5568378, 9337581, 1133018, 965766]","[1092026, 951590, 1053690, 1133018, 1106523]","[951590, 826249, 910032, 1092026, 9337581]","[1082185, 6534178, 1029743, 1053690, 981760]","[6534178, 929668, 1082185, 1106523, 995242]","[1053690, 1133018, 1092026, 951590, 932298]","[951590, 8090521, 1053690, 883404, 8090537]","[951590, 1053690, 5569230, 892008, 883404]",...,"[1133018, 1106523, 1053690, 883404, 951590]","[1133018, 1053690, 1092026, 951590, 1106523]","[1133018, 1092026, 1106523, 5568378, 951590]","[1133018, 1106523, 1092026, 1053690, 910032]","[1133018, 1053690, 1106523, 1092026, 883404]","[1133018, 1092026, 1106523, 1022003, 883404]","[1133018, 1106523, 1053690, 1051069, 1092026]","[1133018, 1053690, 951590, 1092026, 1106523]","[1133018, 1106523, 1053690, 883404, 951590]","[1133018, 1106523, 1053690, 1092026, 951590]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1023720, 1007195, 1051516, 866211, 986912]","[1082185, 1024306, 878996, 1023720, 934639]","[1082185, 878996, 1024306, 1127831, 1023720]","[1082185, 6534178, 1127831, 866211, 981760]","[1023720, 1082185, 878996, 1024306, 965267]","[1024306, 1082185, 1023720, 878996, 1051516]","[1082185, 1024306, 878996, 1023720, 857006]","[1082185, 1024306, 834484, 857006, 1023720]",...,"[1082185, 1007195, 1024306, 878996, 1023720]","[1082185, 878996, 1051516, 866211, 1024306]","[1007195, 1082185, 1023720, 878996, 866211]","[1082185, 878996, 1051516, 1024306, 1023720]","[878996, 1082185, 1051516, 866211, 1007195]","[1082185, 1098248, 1007195, 1023720, 919681]","[1098248, 1007195, 1082185, 1024306, 1023720]","[1082185, 1023720, 878996, 866211, 1024306]","[1082185, 1024306, 878996, 1023720, 866211]","[1082185, 1024306, 1023720, 866211, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[938700, 1082185, 1106523, 826249, 883404]","[1082185, 6944571, 849843, 893018, 1121393]","[1082185, 873527, 828857, 1127831, 1118627]","[1052046, 1013321, 857390, 911812, 1082185]","[1082185, 6534178, 1029743, 995242, 1106523]","[1058997, 1082185, 6944571, 849843, 1126899]","[1082185, 6944571, 849843, 1058997, 989249]","[1082185, 6944571, 849843, 1058997, 989249]",...,"[1082185, 6944571, 1126899, 1058997, 849843]","[1082185, 6944571, 1058997, 828867, 1126899]","[1082185, 6944571, 1126899, 1122358, 826249]","[1082185, 1126899, 6944571, 1058997, 828867]","[1082185, 6944571, 828867, 857390, 1126899]","[1082185, 828867, 6944571, 1127831, 1106523]","

In [46]:
df_res = pd.DataFrame(columns=['parameters', 'presision_at_k'])
i = 0
for column in result.columns[2:]:
    a = round(result.apply(lambda row: precision_at_k(row[column], row['actual']), axis=1).mean(), 3)
    #print(column, a)
    #df = pd.DataFrame([column, a])
    df_res.loc[i] = [column, a]
    i += 1

In [50]:
df_res.sort_values(by='presision_at_k', ascending=False).head(10)

,parameters,presision_at_k
69,als_bm25_factors=170.0_reg=0.01,0.237
68,als_bm25_factors=170.0_reg=0.005179474679231213,0.234
64,als_bm25_factors=170.0_reg=0.0003727593720314938,0.234
63,als_bm25_factors=170.0_reg=0.00019306977288832496,0.234
66,als_bm25_factors=170.0_reg=0.0013894954943731374,0.233
61,als_bm25_factors=150.0_reg=0.01,0.233
52,als_bm25_factors=130.0_reg=0.005179474679231213,0.232
46,als_bm25_factors=130.0_reg=0.0001,0.231
49,als_bm25_factors=130.0_reg=0.0007196856730011522,0.231
60,als_bm25_factors=150.0_reg=0.005179474679231213,0.231


Наилучшие гиперпараметры: factors = 170, lambda = 0.01